**Тема проекта**: Исследование надёжности заёмщиков для разработки скорринговой модели.

**Заказчик** — кредитный отдел банка. 

**Цель исследования**: разобраться, влияет ли семейное положение, количество детей клиента и другие факторы на погашение кредита в срок. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банк

Входные данные от банка — статистика о платёжеспособности клиентов.


# Загрузка данных и изучение общей информации

In [1]:
import pandas as pd

In [3]:
temp = data.copy() 
display(temp.head(5))
print('-'* 25)
display(temp.info())
print('-'* 25)
display(temp.isna().mean())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


-------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

-------------------------


children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

В столбцах days_employed и total_income есть пропущенные значения. Форматы сообтветствуют значениям. Названия столбцов соответствуют принятому стилю.

# Предобработка данных

## Удаление пропусков

В столбце total_income пропущенные значения составляют 10,1%.  
На сумму дохода сильнее всего влияет тип занятости, поэтому целесообразнее всего заполнить пропуски в этом столбце медианным значением по каждому типу из столбца income_type. Например, у человека с типом занятости сотрудник пропуск в столбце total_income должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [4]:
for index in data['income_type'].unique():
    data.loc[(data['income_type'] == index)&(data['total_income'].isna()),'total_income']=\
    data.loc[(data['income_type'] == index)&(data['total_income']>=0),'total_income'].median()
data['total_income'].isna().sum()

0

## Обработка аномальных значений

Проверим данные на артефакты (аномалии). Таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed.  
Заменим все отрицательные значения положительными с помощью метода abs().

In [5]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведим медианное значение трудового стажа days_employed в днях.

In [6]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения не представляется возможным, поэтому оставим их без изменений.

Выведите перечень уникальных значений столбца children.

In [7]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце есть два аномальных значения: -1 и 20 . Удалим строки, в которых встречаются эти аномальные значения. 

In [8]:
data = data[(data['children'] != -1)&(data['children'] != 20)]

## Удаление пропусков (продолжение)

 Заполним пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type.

In [9]:
for index in data['income_type'].unique():
    data.loc[(data['income_type'] == index)&(data['days_employed'].isna()),'days_employed']=\
    data.loc[(data['income_type'] == index)&(data['days_employed']>=0),'days_employed'].median()
data['days_employed'].isna().sum()

0

## Изменение типов данных

Замениv вещественный тип данных в столбце total_income на целочисленный с помощью метода astype().

In [10]:
data['total_income'] = data['total_income'].astype('int')

## Обработка дубликатов

Проверим наличие явных дубликатов и удалим их при наличии.

In [11]:
data.duplicated().sum()

54

In [12]:
data = data.drop_duplicates()

Проверим данные на неявные дубликаты.

In [13]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [14]:
qual_columns = ['education', 'family_status', 'gender', 'income_type', 'purpose']
for i in qual_columns:
    print(i, data[i].sort_values().unique())
    print('-'* 25)

education ['ВЫСШЕЕ' 'Высшее' 'НАЧАЛЬНОЕ' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Начальное'
 'Неоконченное высшее' 'СРЕДНЕЕ' 'Среднее' 'УЧЕНАЯ СТЕПЕНЬ'
 'Ученая степень' 'высшее' 'начальное' 'неоконченное высшее' 'среднее'
 'ученая степень']
-------------------------
family_status ['Не женат / не замужем' 'в разводе' 'вдовец / вдова' 'гражданский брак'
 'женат / замужем']
-------------------------
gender ['F' 'M' 'XNA']
-------------------------
income_type ['безработный' 'в декрете' 'госслужащий' 'компаньон' 'пенсионер'
 'предприниматель' 'сотрудник' 'студент']
-------------------------
purpose ['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомобиля' 'на покупку своего автомобиля'
 'на проведение свадьбы' 'недвижимость' 'образование' 'операции с жильем'
 'операции с коммерческой недвижимостью' 'операции с недвижимостью'
 'операции со своей недвижимостью' 'п

Обработаем неявные дубликаты в столбце education. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [15]:
data['education'] = data['education'].str.lower()

Неявные дубликаты в столбце purpose пока не будем трогать, обрабатаем их по-другому чуть позже.

## Категоризация данных

На основании диапазонов доходов, указанных ниже, создадим столбец total_income_category с категориями:

0–30000 — 'E';  
30001–50000 — 'D';  
50001–200000 — 'C';  
200001–1000000 — 'B';  
1000001 и выше — 'A'.  

Для этого создадим собственную функцию с именем categorize_income().

In [16]:
def categorize_income(income):
    if 0 < income <=30000:
        return 'E'
    if 30001 < income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <=1000000:
        return 'B'
    if income >= 1000001:
        return 'A'

In [17]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

- 'операции с автомобилем',  
- 'операции с недвижимостью',  
- 'проведение свадьбы',  
- 'получение образования'.  

Для этого создадим собственную функцию с именем categorize_purpose(). егорию.

In [18]:
def categorize_purpose(purpose):
    if 'образ' in purpose:
        return 'получение образования'
    if 'авто' in purpose: 
        return 'операции с автомобилем'
    if 'свад' in purpose:
        return 'проведение свадьбы'
    return 'операции с недвижимостью'

In [19]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

# Исследовательский анализ данных.

## Определение зависимости между количеством детей и возвратом кредита в срок.

In [20]:
data_children = data.groupby('children').agg({'debt':['sum','count','mean']}).sort_values(by=('debt','mean'),ascending=False)


In [21]:
data_children.columns = ['sum','count','debt_share']


In [22]:
data_children['category_share'] = round(data_children['count']*100/data_children['count'].sum(),2)
data_children

,sum,count,debt_share,category_share
children,,,,
4,4,41,0.097561,0.19
2,194,2052,0.094542,9.61
1,444,4809,0.092327,22.53
3,27,330,0.081818,1.55
0,1063,14107,0.075353,66.08
5,0,9,0.000000,0.04


Вывод: Семьи с 5 детьми гасят кредиты в срок без просрочек.  

Заемщики без детей составляют самую многочисленную группу заемщиков(66% от общего количества заемщиков) и доля просрочек в этой группе ниже всех - 7,54%.  

Заемщики с 1-2 детьми допускают просрочки чуть чаще, доля просрочек составляет 9,23% и 9,45%, соответственно. А их доля в общем объеме заемщиков - 22,53% и 9,61%, соответственно.  

Доля должников среди заемщиков с 3 детьми - 8,18%, с 4 детьми - 9,75%, с 5 детьми - 0,0%.  

Т.о., видно, что появление детей влечет за собой дополнительную финансовую нагрузку и заемщики с детьми чаще допускают просрочки, но незначительно.  Разница в доле просрочки колеблется в пределах 1-2%.

Также стоит отметить, что группа с 3, 4 и 5 детьми очень малочисленны и составляют всего 1,55%, 0,19% и 0,4%, соответственно,  от общего количества заемщиков, поэтому эти группы существенно не влияют на долю возвратов кредитов в целом и их можно не принимать в расчет. 

## Определение зависимости между семейным положением и возвратом кредита в срок.

In [23]:
data_family_status = data.groupby('family_status').agg({'debt':['sum','count','mean']}).sort_values(by=('debt','mean'),ascending=False)


In [24]:
data_family_status.columns = ['sum','count','debt_share']

In [25]:
data_family_status['category_share'] = round(data_family_status['count']*100/data_family_status['count'].sum(),2)
data_family_status

,sum,count,debt_share,category_share
family_status,,,,
Не женат / не замужем,273,2796,0.097639,13.10
гражданский брак,385,4146,0.092861,19.42
женат / замужем,927,12266,0.075575,57.46
в разводе,84,1189,0.070648,5.57
вдовец / вдова,63,951,0.066246,4.45


Меньше всего доля просрочек у вдовцов и людей в разводе: 6,62% и 7,06% соответственно. Можно предположить, что люди, оставшиеся одни, очень хорошо взвешивают свои решения прежде, чем взять на себя какие-либо обязательства, т.к. понимают, что нужно рассчитывать только на себя, и имеют жизненный опыт. Их доля в общем количестве заемщиков составляет 4,45% и 5,57%, соответственно (примерно 1/10 от общего числа заемщиков).

Доля просрочек у людей, состоящих в браке(эта группа составляет 57.46% от общего количества заемщиков- больше 1/2) - 7,56%.  Она чуть выше, чем у предыдущей группы. Можно предположить, что у семейных заемщиков чуть чаще возникают непредвиденные траты или меняются финансовые приоритеты.

Чаще всего допускают просрочки люди не в браке или состоящие в гражданском браке: 9,76% и 9,31%. Их доли в общем числе заемщиков - 13,10% и 19,42%, соотвественно(в общей сложности примерно 1/3 от общего числа заемщиков). Т.е., это та группа людей, среди которых больше молодых людей, еще не готовых брать на себя серьезную ответственность. 

## Определение зависимости между уровнем дохода и возвратом кредита в срок.

In [26]:
data_income_category = data.groupby('total_income_category').agg({'debt':['sum','count','mean']}).sort_values(by=('debt','mean'),ascending=False)

In [27]:
data_income_category.columns = ['sum','count','debt_share']

In [28]:
data_income_category['category_share'] = round(data_income_category['count']*100/data_income_category['count'].sum(),2)
data_income_category

,sum,count,debt_share,category_share
total_income_category,,,,
E,2,22,0.090909,0.10
C,1353,15938,0.084891,74.66
A,2,25,0.080000,0.12
B,354,5014,0.070602,23.49
D,21,349,0.060172,1.63


0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.

Вывод:  
Чаще всего допускают просрочки люди с минимальным уровнем дохода - до 30 000р., доля просрочки - 9,09%. Но их всего 0,1% от общего количества клиентов, также, как и людей с доходом выше 1 000 000, у которых доля просрочки 8,00%.  

Реже всего допускают просрочки люди с доходом от 30 000 до 50 000, у них доля просрочки - 6,02% (но их тоже всего 1,63% от общего количества заемщиков).  
''.

Лучше рассматривать группу с доходом от 50 000 до 200 000, их 74,66% от общего количества заемщиков, а доля просрочки в этой группе составляет 8,50%.  

Доля просрочки у людей с доходом от 200 000 до 1 000 000 - 7,06% (их 23,49% от общего количества заемщиков).  

Если смотреть по самым многочисленным группам, то чем выше доход, тем ниже уровень просрочек.

## Определение, как влияют разные цели кредита на его возврат в срок.

In [29]:
data_purpose = data.groupby('purpose_category').agg({'debt':['sum','count','mean']}).sort_values(by=('debt','mean'),ascending=False)

In [30]:
data_purpose.columns = ['sum','count','debt_share']

In [31]:
data_purpose['category_share'] = round(data_purpose['count']*100/data_purpose['count'].sum(),2)
data_purpose

,sum,count,debt_share,category_share
purpose_category,,,,
операции с автомобилем,400,4281,0.093436,20.05
получение образования,369,3989,0.092504,18.69
проведение свадьбы,183,2324,0.078744,10.89
операции с недвижимостью,780,10754,0.072531,50.37


Ниже всего уровень просрочек по кредитам на недвижимость - 7,25% (доля таких кредитов - 50,37% в общем портфеле заемщиков),  
на проведение свадеб - 7,87% (10,89% от общего количества кредитов).   
Максимальный уровень просрочек по кредитам на образование и операции с автомобилем. Он примерно одинаковый по этим категориям и составляет 9,3% (доля этих категорий в общем портфеле тоже примерно одинаковая: 18,69% и 20.05%)

## Возможные причины появления пропусков в исходных данных.

Пропуски в данных содержатся в столбцах 'days_employed' и 'total_income'. Можно предположить, что это созаемщики, являющиеся супругами основных заемщиков, доход которых банк не рассматривает для расчета дохода (жены в декрете или супруги, которые не могут подтвердить официальный доход, не работают или учатся). Если эта версия верна, то тогда лучше исключить данные по этим заемщикам, а не заполнять их медианными значениями.

# Выводы

Отчет об исследовании статистики платёжеспособности клиентов. Результаты могут быть использованы для  построения модели кредитного скорринга, оценивающей платежеспособность потенциального заёмщика.  

1. Для исследования взяты входные данные от банка — статистика о платёжеспособности 21.5 тысячи клиентов со следующими 10 параметрами: количество детей, общий трудовой стаж, возраст, уровень образования, семейное положение, пол клиента, тип занятости, наличие просрочек, ежемесячный доход и цель получения кредита.

2. Проведена предобработка данных.

В данных были выявлены пропуски и некорректные значения и предприняты меры для их устранения:
- отработаны отрицательные значения трудового стажа путем приведения их к положительному значению,
- в 10% значений в графах ежемесячного дохода и общего стажа заемщиков были допущены пропуски. Для устранения этой проблемы в столбце общего дохода и общего стажа пропущенные значения заменены медианными значениями, рассчитанными для соответствующего типа занятости. Медианные значения позволяют избежать искажения данных при сильно отличающихся крайних значениях.
- удалены аномальные значения (-1 и 20) в столбце «количество детей»,
- удалены явные дубликаты,
- отработаны неявные дубликаты в столбце  «уровень образования» путем приведения значений к нижнему регистру.

Проведена категоризация данных по столбцу «общий доход» путем создания диапазонов уровня дохода клиентов и присвоения им 
буквенного обозначения:  
•0–30000 — 'E';  
•30001–50000 — 'D';  
•50001–200000 — 'C';  
•200001–1000000 — 'B';  
•1000001 и выше — 'A'.  

Проведена категоризация данных по столбцу «цель кредита» путем объединения их в следующие категории:  
• операции с автомобилем,  
• операции с недвижимостью,  
• проведение свадьбы,  
• получение образования.  

3. После проведения предобработки данных был сделан анализ зависимости возврата заемщиками кредита в срок от 4 факторов:
количества детей, семейного положения, уровня дохода и цели кредита. 
Получены следующие выводы:

3.1. Семьи с 5 детьми гасят кредиты в срок без просрочек.

Заемщики без детей составляют самую многочисленную группу заемщиков(66% от общего количества заемщиков) и доля просрочек в этой группе ниже всех - 7,54%.

Заемщики с 1-2 детьми допускают просрочки чуть чаще, доля просрочек составляет 9,23% и 9,45%, соответственно. А их доля в общем объеме заемщиков - 22,53% и 9,61%, соответственно.

Доля должников среди заемщиков с 3 детьми - 8,18%, с 4 детьми - 9,75%, с 5 детьми - 0,0%.

Т.о., видно, что появление детей влечет за собой дополнительную финансовую нагрузку и заемщики с детьми чаще допускают просрочки, но незначительно. Разница в доле просрочки колеблется в пределах 1-2%.

Также стоит отметить, что группа с 3, 4 и 5 детьми очень малочисленны и составляют всего 1,55%, 0,19% и 0,4%, соответственно, от общего количества заемщиков, поэтому эти группы существенно не влияют на долю возвратов кредитов в целом и их можно не принимать ивается. 

3.2. Зависимость от семейного положения также просл
Меньше всего доля просрочек у вдовцов и людей в разводе: 6,62% и 7,06% соответственно. Можно предположить, что теряя финансовую поддержку своих партнеров, они очень взвешенно подходят к вопросу оценки своих финансовых возможностей по обслуживанию кредита. Их доля в общем количестве заемщиков составляет 4,45% и 5,57%, соответственно (примерно 1/10 от общего числа заемщиков).

Доля просрочек у людей, состоящих в браке(эта группа составляет 57.46% от общего количества заемщиков- больше 1/2) - 7,56%. Она чуть выше, чем у предыдущей группы. Можно предположить, что у семейных заемщиков чуть чаще возникают непредвиденные траты или меняются финансовые приоритеты.

Чаще всего допускают просрочки люди не в браке или состоящие в гражданском браке: 9,76% и 9,31%. Их доли в общем числе заемщиков - 13,10% и 19,42%, соотвественно(в общей сложности примерно 1/3 от общего числа заемщикоа
заемщиков). Можно предположить, что из-за своего молодого возраста или отсутствия жизненного опыта они более егкомысленно 
относятся к своим обязательств
ам по кредиту.
3.3. Если оценивать заемщиков по уровню дохода, то чаще всего допускают просрочки люди с минимальным уровнем доход(т.к. их уровень дохода едва позволяет им закрывать свои базовые потребности)

Чаще всего допускают просрочки люди с минимальным уровнем дохода - до 30 000р., доля просрочки - 9,09%. Но их всего 0,1% от общего количества клиентов, также, как и людей с доходом выше 1 000 000, у которых доля просрочки 8,00%.

Реже всего допускают просрочки люди с доходом от 30 000 до 50 000, у них доля просрочки - 6,02% (но их тоже всего 1,63% от общего количества заемщиков Можно предположить, что эта категория берет кредиты только в случае острой необходимости, тщательно взвешивая перед этим свои финансовые возможности.)'.

Лучше рассматривать группу с доходом от 50 000 до 200 000, их 74,66% от общего количества заемщиков, а доля просрочки в этой группе составляет 8,50%.

Доля просрочки у людей с доходом от 200 000 до 1 000 000 - 7,06% (их 23,49% от общего количества заемщиков).

Если смотреть по самым многочисленным группам, то чем выше доход, тем ниже уровень прое уровень просрочек. 

3.4. Рассматривая зависимость от цели кредитниже всего уровень просрочек по кредитам на недвижимость - 7,25% (доля таких кредитов - 50,37% в общем портфеле заемщиков). Как правило, в данном случае люди берут кредит на то жилье, в котором они будут проживать и риск потерять это жилье из-за неплатежей, дисциплинирует заемщиков.У кредитов 
на проведение свадебдоля просрочек составляет- 7,87% (10,89% от общего количества кредитов)  

Максимальный уровень просрочек по кредитам на образование и операции с автомобилем. Он примерно одинаковый по этим категориям и составляет 9,3% (доля этих категорий в общем портфеле тоже примерно одинаковая: 18,69% и 20.05%)ковая: 18,7% и 20.06%).

Т.о., можно сделать вывод, что скорринговая модель, оценивающая вероятность погашения заемщиками кредита в срок, должна быть многофакторной и учитывать не только уровень дохода потенциальных клиентов, но и их семейное положение, количество детей и цель, на которую берется кредит.
 